
## Set-up for the project environment for GizmoBox Data Lakehouse

1. Access the container gizmobox


In [0]:
%fs ls 'abfss://gizmobox@deacourseextdld.dfs.core.windows.net/Landing'

In [0]:
%sql
create external location if not exists dea_course_ext_dl_gizmobox
  url 'abfss://gizmobox@deacourseextdld.dfs.core.windows.net/'
  with (storage credential dea_course_ext_sc)
  comment 'External location for Gizmo Box data lakehouse;'

### 3. Create the Gizmobox- Catalog



In [0]:
%sql

show catalogs;

In [0]:
%sql

create catalog if not exists gizmobox
  managed location 'abfss://gizmobox@deacourseextdld.dfs.core.windows.net/' -- If not specified then the default catalog is used. which is root
  comment 'This is the catalog for Gizmo Box';


#### 4. Create Schemas

1. Landing
2. Bronze
3. Silver
4. Gold

In [0]:
%sql
SELECT current_catalog();

In [0]:
%sql
USE CATALOG gizmobox;
    
SHOW SCHEMAS;

In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS Landing
  COMMENT 'This is the landing schema for Gizmo Box'
  MANAGED LOCATION 'abfss://gizmobox@deacourseextdld.dfs.core.windows.net/Landing';
    
CREATE SCHEMA IF NOT EXISTS bronze
  COMMENT 'This is the bronze schema for Gizmo Box'
  MANAGED LOCATION 'abfss://gizmobox@deacourseextdld.dfs.core.windows.net/bronze';
    
CREATE SCHEMA IF NOT EXISTS silver
  COMMENT 'This is the silver schema for Gizmo Box'
  MANAGED LOCATION 'abfss://gizmobox@deacourseextdld.dfs.core.windows.net/silver';
    
CREATE SCHEMA IF NOT EXISTS gold
  COMMENT 'This is the gold schema for Gizmo Box'
  MANAGED LOCATION 'abfss://gizmobox@deacourseextdld.dfs.core.windows.net/gold';
    
SHOW SCHEMAS;

### 5. Create Volume


In [0]:
%sql
USE CATALOG gizmobox;
USE SCHEMA landing;

CREATE EXTERNAL VOLUME IF NOT EXISTS operational_data
  COMMENT 'This is the operational data volume for Gizmo Box'
  LOCATION 'abfss://gizmobox@deacourseextdld.dfs.core.windows.net/Landing/operational_data';





In [0]:
%fs ls /Volumes/gizmobox/landing/operational_data

Querying the data 

In [0]:
%sql
-- Single File
select * from json.`/Volumes/gizmobox/landing/operational_data/customers/customers_2024_10.json`

In [0]:
%sql
--- Querying multiple files
select * from json.`/Volumes/gizmobox/landing/operational_data/customers/customers_2024_*.json`

In [0]:
%sql
--- Querying entire folder
SELECT * FROM JSON.`/Volumes/gizmobox/landing/operational_data/customers`

In [0]:
%sql

--- To enable data auditing to know which records are from which file
select _metadata.file_path AS file_path,
  * from json.`/Volumes/gizmobox/landing/operational_data/customers`

#### 5. Register Files in Unity Catalog using Views


In [0]:
%sql
use catalog gizmobox;
use schema bronze;

create or replace view v_customers
as 
select *, _metadata.file_path as file_path from json.`/Volumes/gizmobox/landing/operational_data/customers`
    


In [0]:
%sql
select * from gizmobox.bronze.v_customers;

### Creating Temporary View

This view is only session based, once the session ends, the temp view will be gone. 

### Global Temporary View

This view is cluster based, it lives until the cluster runs and deletes when cluster is stopped.

In [0]:
%sql
create or replace temporary view tv_customers
as
select *, _metadata.file_path as file_path from json.`/Volumes/gizmobox/landing/operational_data/customers`
    


In [0]:
%sql
select * from tv_customers

In [0]:
%sql
create or replace global temporary view gtv_customers
as
select *, _metadata.file_path as file_path from json.`/Volumes/gizmobox/landing/operational_data/customers`

In [0]:
%sql
select * from global_temp.gtv_customers;